<img align="left" src = https://dirac.astro.washington.edu/wp-content/uploads/2022/03/DIRAC_logo-purple-text_transparent-bckg_ALPHA-1-1.png
 width=190 style="padding: 10px" alt="DiRAC Logo">
<br><b>Create a Table of White Dwarfs ("acquire_WDs.ipynb")</b> <br>
Contact author: Bob Abel <br>
Last verified to run: 2023-11-07 <br>
LSST Science Piplines version: Weekly 2023_37 <br>
Container size: medium <br>
Targeted learning level: beginner? <br>
<figcaption align = "left"><b>&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;DiRAC Institute</b></figcaption>


Objective:
Acquire white dwarfs from two different data bases and merge with Gaia data<br>Description:
Color-magnitude models for DA, DB and DC white dwarfs are prepared using two publicly available white dwarf catalogs and Gaia and SDSS data<br>LSST Data Products:
Dustmaps.<br>
Credits:
Tina Adair, Jennifer Soebeck, Douglas Tucker, Željko Ivezić, Brian Yanny, Allyn Smith, Allan Jackson, literally every single contributing author to the Rubin Stack Club, DP0 Tutorial, and Delegate Contribution Jupyter Notebooks, and Jake Vanderplas.

### <span style='color:blue'> TABLE OF CONTENTS </span>
#### <span style='color:blue'>     I. Import Modules </span>
#### <span style='color:blue'>     II. Load and Prep the Montreal White Dwarf Database </span>
#### <span style='color:blue'>     III. Load and Prep the Warwick (Fusillo et al. 2021) Database </span>
#### <span style='color:blue'>     IV. Merge the Two Databases' Spectral Types, Based on Their ID's </span>
#### <span style='color:blue'>     V. Load and Prep the Gaia DR3 (Bailer-Jones et al 2022) Database and Merge with the Spectral Types </span>
#### <span style='color:blue'>     VI. Filter Final Table by Magnitude, Add Galactic Coordinates  </span>
#### <span style='color:blue'>     VII. Save Final Table to 'MontWarWDs.csv' </span>
    

### <span style='color:blue'> I. Import Modules

In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt

# Import Rubin Pipeline extinction maps
from dustmaps.sfd import SFDQuery


Overriding default configuration file with /opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/dustmaps_cachedata/gbb0a0c949e+81bc2a20b4/config/.dustmapsrc


### <span style='color:blue'> II. Load  the [Montreal  White Dwarf DataBase](https://www.montrealwhitedwarfdatabase.org/), make object ID's compatible with the Warwick (Fusillo et al.) Catalog and save only DA, DB and DC white dwarfs.</span>

In [2]:
# Load the (slightly filtered) Montreal White Dwarf Database
df_MontWD = pd.read_csv('MWDD-export.csv', low_memory=False)

# Get rid of some spaces that make it hard to read
df_MontWD.replace('', np.nan, inplace=True)
df_MontWD.replace('      ', np.nan, inplace=True)

# Get rid of the "GaiaEDR3" or "GaiaDR2" titles to turn df_MontWD['name'] 
# into a df_MontWD['source_id_2'] that's compatible with the Warwick data.
# Convert all the Gaia Source ID's to strings for compatibility.

GaiaEDR3 = []
for i in range(len(df_MontWD)):
    if df_MontWD['name'][i].startswith('Gaia'):
        GaiaEDR3.append(int(df_MontWD['name'][i][9:]))
    else:
        GaiaEDR3.append('NaN')    

source_id_2 = []
for i in range(len(df_MontWD)):
    source_id_2.append(str(GaiaEDR3[i]))
df_MontWD['source_id_2'] = source_id_2


# Limit the database to DA, DB and DC WD's
df_MontWD = df_MontWD[(df_MontWD['spectype'] == 'DA') | (df_MontWD['spectype'] == 'DB') | 
                     (df_MontWD['spectype'] == 'DC') ]

df_MontWD = df_MontWD.drop(columns=['wdid', 'name', 'wdname', 'icrsra', 'icrsdec', 'teff',
       'logg', 'parallax', 'Dpc', 'G', 'DG', 'BP', 'RP'], axis=1)

df_MontWD = df_MontWD.reset_index()


### <span style='color:blue'> III. The Montreal WD Database for DA, DB and DC WD's only is ready to merge. Load the Warwick ([Fusillo et al. 2021](https://arxiv.org/abs/2106.07669)) Catalog, make object ID column titles compatible with the Montreal Catalog and save only DA, DB and DC white dwarfs.

In [3]:
df_WDcat = pd.read_csv('gaia-sdss_white_dwarf_catalogue_v2.1.csv')

# Rename columns to match Montreal White Dwarf Database columns.
df_WDcat = df_WDcat.rename(columns={'white_dwarf_name':'WDJname'}, errors='raise')
df_WDcat = df_WDcat.rename(columns={'spectral_class':'spectype'}, errors='raise')

# Save only DA, DB and DC white dwarfs.
df_WDcat = df_WDcat[(df_WDcat['spectype'] == 'DA') | (df_WDcat['spectype'] == 'DB') | 
                     (df_WDcat['spectype'] == 'DC') ]

df_WDcat = df_WDcat.drop(columns=['WDJname', 'SDSS_name', 'SDSS_ra', 'SDSS_dec', 
        'S/N', 'umag', 'e_umag', 'gmag', 'e_gmag', 'rmag', 'e_rmag', 'imag', 
        'e_imag', 'zmag', 'e_zmag', 'Plate', 'mjd', 'fiberID'], axis=1)

df_WDcat = df_WDcat.reset_index()
df_WDcat.columns

Index(['index', 'source_id_2', 'spectype'], dtype='object')

### <span style='color:blue'> IV. Merge the Gaia ID's and spectral types for the two databases into one table("df").


In [4]:
# Collect the Gaia ID's and spectral types for both WD databases into one list
# and open up a new DataFrame.
source_id_2 = []
spectype = []
for i in range(len(df_MontWD)):
    source_id_2.append(str(df_MontWD['source_id_2'][i]))
    spectype.append(df_MontWD['spectype'][i])

for i in range(len(df_WDcat)):
    source_id_2.append(str(df_WDcat['source_id_2'][i]))
    spectype.append(df_WDcat['spectype'][i])

# Make a new DataFrame with the combined WD catalogs and an ID column that matches
# the name of the Gaia catalog's column:    
df = pd.DataFrame()
df['GaiaEDR3'] = source_id_2
df['spectype'] = spectype

df.drop_duplicates('GaiaEDR3')

del df_MontWD, df_WDcat

### <span style='color:blue'> V. Load the Gaia ([Bailer-Jones et al 2021](https://arxiv.org/pdf/2012.05220.pdf)) database, modify the object ID's for compatibility and merge it with the spectypes, based on the object ID. From [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=J/MNRAS/508/3877/maincat), get Gaia data with geometric and photogeometric distances. </span>

In [5]:
# An object to convert a votable file to a Pandas table
from astropy.io.votable import parse

def votable_to_pandas(votable_file):
    votable = parse(votable_file)
    table = votable.get_first_table().to_table(use_names_over_ids=True)
    return table.to_pandas()

In [6]:
# Download the Gaia data from VizieR as a votable file and make a Pandas table
df_Gaia = votable_to_pandas('vizier_votable.vot')

# Convert all GAIA ID's to strings for compatibility
GaiaEDR3 = []
for i in range(len(df_Gaia)):
    GaiaEDR3.append(str(df_Gaia['GaiaEDR3'][i]))

df_Gaia['GaiaEDR3'] = GaiaEDR3
df_Gaia = df_Gaia.reset_index()

df_merge = pd.merge(df_Gaia, df, on='GaiaEDR3')

### <span style='color:blue'> VI. Merging is complete. Drop any duplicate rows, filter magnitudes (generally 5-22, but r is 15-19) and add galactic coordinates. Keep b > $15^{o}$ to stay out of the bulge.

In [7]:
# define a function to calculate galactic coordinates
def get_galactic_coord(row):
    c = SkyCoord(ra=row['RA_ICRS']*u.degree, dec=row['DE_ICRS']*u.degree, frame='icrs')
    row['l'] = c.galactic.l.degree
    row['b'] = c.galactic.b.degree
    return row



In [8]:
#Drop any duplicate rows
df_merge = df_merge.drop_duplicates('GaiaEDR3')

# Eliminate non-positive magnitudes and high SNR magnitudes.
df_merge = df_merge[ ( df_merge['umag'] > 5.) & (df_merge['umag'] < 22.)
                 & (df_merge['gmag'] > 5.) & (df_merge['gmag'] < 22.)
                 & (df_merge['rmag'] > 15.) & (df_merge['rmag'] < 19.)
                 & (df_merge['imag'] > 5.) & (df_merge['imag'] < 22.)
                 & (df_merge['zmag'] > 5.) & (df_merge['zmag'] < 22.)]


# apply the galactic coordinate calculation to each row using the apply method
# and limit WD's to b > 15 degrees to stay out of the bulge.
df_merge = df_merge.apply(get_galactic_coord, axis=1)
df_merge = df_merge[(np.abs(df_merge['b']) > 15.)]
df_merge = df_merge[df_merge['Pwd'] >= 0.9]
df_merge = df_merge.drop(columns=['WDJname', '_RA.icrs', '_DE.icrs'], axis=1)
df_merge = df_merge.dropna()
df_merge = df_merge.reset_index()

### <span style='color:blue'> VII. Check the data columns and create an output file ('MontWarWDs.csv'). We'll use this file to calculate absolute magnitudes and create white dwarf color-Magnitude models in "model_WDs.ipynb".

In [10]:
print('Number of Rows in Table: %d' %len(df_merge))
print('Spectral Types: ', df_merge.spectype.unique())
print()
print('WD Type    #')
print(df_merge.spectype.value_counts())

Number of Rows in Table: 7077
Spectral Types:  ['DA' 'DC' 'DB']

WD Type    #
spectype
DA    5870
DB     702
DC     505
Name: count, dtype: int64


In [11]:
df_merge.columns

Index(['level_0', 'index', 'GaiaEDR3', 'RA_ICRS', 'DE_ICRS', 'Plx', 'Pwd',
       'PM', 'pmRA', 'pmDE', 'Gmag', 'BPmag', 'RPmag', 'TeffH', 'e_TeffH',
       'loggH', 'e_loggH', 'MassH', 'TeffHe', 'e_TeffHe', 'loggHe', 'e_loggHe',
       'MassHe', 'Teffmix', 'e_Teffmix', 'loggmix', 'e_loggmix', 'rgeo',
       'b_rgeo', 'B_rgeo', 'rpgeo', 'b_rpgeo', 'B_rpgeo', 'SDSS12', 'umag',
       'gmag', 'rmag', 'imag', 'zmag', 'recno', 'spectype', 'l', 'b'],
      dtype='object')

In [12]:
df_merge.to_csv('MontWarWDs.csv')